In [8]:
from core.utils import Tibanna
from core import ff_utils
from invoke import run
import time
from datetime import datetime

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)

intr = "intermediate file"

Converter = {
    "bwa-mem":{"bam":intr},
    "pairsam-parse-sort":{"pairsam":intr},
    "pairsam-merge":{"pairsam":intr},
    "pairsam-markasdup":{"pairsam":intr},
    "pairsam-filter":{"pairs":intr,"bam":"alignment"},
    "addfragtopairs":{"pairs":intr},
    "pairs-patch":{"pairs":"contact list"},
    "hi-c-processing-partb": {"hic": "contact matrix","cool":intr, "pairs":intr},
    "hi-c-processing-partc": {"mcool":"contact matrix", "normvector_juicerformat":"juicebox norm vector"}
}

ff = ff_utils.fdn_connection(key=tibanna.ff_keys)
all_aws = ff_utils.get_metadata('workflow-runs-awsem', connection=ff)['@graph']

hi-c-processing-partb run 2017-09-18 09:57:57.367927
[{u'secondary_file_formats': [u'pairs_px2'], u'extension': u'.pairs.gz', u'format': u'pairs', u'workflow_argument_name': u'output_pairs', u'value': u'/files-processed/4DNFIXX3YX2S/', u'upload_key': u'da9b5012-a7e6-49ec-93e2-a1e56716bafb/4DNFIXX3YX2S.pairs.gz', u'secondary_file_extensions': [u'.pairs.gz.px2'], u'type': u'Output processed file', u'extra_files': [{u'status': u'to be uploaded by workflow', u'uuid': u'da9b5012-a7e6-49ec-93e2-a1e56716bafb', u'file_format': u'pairs_px2', u'accession': u'4DNFIXX3YX2S', u'filename': u'4DNFIXX3YX2S', u'upload_key': u'da9b5012-a7e6-49ec-93e2-a1e56716bafb/4DNFIXX3YX2S.pairs.gz.px2', u'href': u'/da9b5012-a7e6-49ec-93e2-a1e56716bafb/@@download/4DNFIXX3YX2S.pairs.gz.px2'}]}, {u'extension': u'.cool', u'format': u'cool', u'workflow_argument_name': u'out_cool', u'value': u'/files-processed/4DNFIZ4CHMSU/', u'upload_key': u'c4d73780-663b-47f6-baf7-1e7fdde32f7a/4DNFIZ4CHMSU.cool', u'type': u'Output proce

In [24]:
my_wfrs = Converter.keys()
print my_wfrs

out_n = "This is an output of the current Hi-C processing pipeline draft. NOTE: The pipeline is under development and the file may be superseded. If you use this file, please check back later to confirm its validity. Report any issues to support@4dnucleome.org."
int_n = "This is an intermediate file in the HiC processing pipeline. NOTE: This file may be removed without notice."
success = 0
patch_problem = 0
deleted = 0
for wfr_resp in all_aws:
    if wfr_resp['status'] == 'deleted':
        deleted +=1
        continue
        
    wfr_name = wfr_resp['display_title'].split(' run')[0]
    if wfr_name in my_wfrs:
        outputs = wfr_resp.get('output_files')
        for output in outputs:
            
            form = output['format']
            file_id = output['value']
            f_type =  Converter[wfr_name][form]
            
            desc = ""
            if f_type == "intermediate file":
                desc = int_n
            else:
                desc = out_n
                
            patch_data = {"description": desc, 'file_type': f_type}
            try:
                ff_utils.patch_metadata(patch_data, obj_id=file_id ,connection=ff)
                success += 1
                print success
            except Exception as e:
                print e
                print
                patch_problem += 1
            
print success
print patch_problem
print deleted


['pairsam-filter', 'pairsam-parse-sort', 'addfragtopairs', 'pairs-patch', 'bwa-mem', 'pairsam-merge', 'pairsam-markasdup', 'hi-c-processing-partb', 'hi-c-processing-partc']
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234